## 6.3 Conversation Buffer Memory using ECEL

<div style="text-align: right"> Initial issue : 2025.05.12 </div>
<div style="text-align: right"> last update : 2025.05.12 </div>

ECEL에 chat_history를 추가하는 방법(또 다른 버전)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [3]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [4]:
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

/tmp/ipykernel_72874/2844409624.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [5]:
print(memory.load_memory_variables({}))

{'chat_history': []}


In [6]:
print(memory.load_memory_variables({})["chat_history"])

[]


`RunnablePassthrough.assign`을 사용하여 `chat_history` 변수에 `memory.load_memory_variables` 함수의 결과를 할당.  
이 결과에서 `chat_history` 키에 해당하는 값을 추출

In [7]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key 와 동일하게 입력
)

In [8]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [9]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

프롬프트 

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain 생성

In [11]:
chain = runnable | prompt | llm

In [12]:

content = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 홍길동입니다."}).content
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 홍길동입니다."}, {"ai": content}
)
print(content)

안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?


In [13]:
content = chain.invoke({"input": "내이름이 뭐라고?"}).content
memory.save_context(
    {"human": "내이름이 뭐라고?"}, {"ai": content}
)
print(content)

당신의 이름은 홍길동이라고 하셨습니다. 맞나요?
